# Создание различных ML моделей для определения спам сообщений

## Импорт необходимых библиотек

> Ячейки с установкой библиотек были приведены к типу `Raw`, чтобы они не выполнялись при автоматическом запуске всех ячеек.

In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from scipy.sparse import hstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import pickle

In [2]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import nltk
from nltk.corpus import stopwords

In [3]:
import sys
sys.path.append("..")  # Добавление корневой папки в список путей поиска модулей.
from utils.preprocessing import count_tags, count_links, count_emojis, count_newlines, count_whitespaces, count_newlines, replace_tags, replace_links, remove_emojis, preprocess_text

2025-09-03 02:47:30,132 - INFO - Логгер успешно настроен. Логи записываются в файл: logs/2025-09-03_02-47-30.log


## Чтение обработанного датасета

In [4]:
df = pd.read_csv('../data/preprocessed.csv', index_col=0)

In [5]:
df.head()

,text,label,emojis,newlines,whitespaces,links,tags,text_preprocessed
0,Добрый день! Отличается ли перечень необходимы...,0,0,0,0,0,0,добрый день отличается ли перечень необходимых...
1,Узбекистан. Рассматриваются обе формы,0,0,0,0,0,0,узбекистан рассматриваются обе формы
2,"Здравствуйте, а как проходит поступление после...",0,0,0,0,0,0,здравствуйте а как проходит поступление после ...
3,Спасибо большое за ответ!,0,0,0,0,0,0,спасибо большое за ответ
4,"Здравствуйте, а когда будет день открытых двер...",0,0,0,0,0,0,здравствуйте а когда будет день открытых двере...


## Подготовка выборок

In [6]:
texts = list(df['text_preprocessed'])  # Тексты
labels = list(df['label'])  # Метки (целевые значения)

In [7]:
additional_features = list(df[['emojis', 'newlines', 'whitespaces', 'links', 'tags']].values)  # Дополнительные числовые признаки

### Разделение на обучающую и тестовую выборки

In [8]:
X_train_text, X_test_text, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

In [9]:
X_train_num, X_test_num = train_test_split(
    additional_features, test_size=0.2, random_state=42, stratify=labels
)

### Подготовка стоп-слов

Загрузка английских и русских стоп-слов (`nltk`)

In [10]:
nltk.download('stopwords')
russian_stopwords = set(stopwords.words('russian'))
english_stopwords = set(ENGLISH_STOP_WORDS)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\overklassniy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Объединение русских и английских стоп-слов

In [11]:
combined_stopwords = list(russian_stopwords.union(english_stopwords))

In [12]:
combined_stopwords

['towards',
 'anyway',
 'same',
 'сейчас',
 'he',
 'seem',
 'все',
 'whose',
 'nor',
 'never',
 'иногда',
 'hereafter',
 'system',
 'так',
 'toward',
 'thereupon',
 'several',
 'can',
 'almost',
 'behind',
 'however',
 'лучше',
 'hence',
 'last',
 'next',
 'everything',
 'anything',
 'с',
 'would',
 'already',
 'during',
 'onto',
 'few',
 'тем',
 'some',
 'он',
 'together',
 'should',
 'ж',
 'были',
 'даже',
 'across',
 'сам',
 'был',
 'been',
 'rather',
 'found',
 'только',
 'про',
 'off',
 'herself',
 'twenty',
 'that',
 'без',
 'therefore',
 'нельзя',
 'кто',
 'now',
 'go',
 'со',
 'cry',
 'ни',
 'я',
 'forty',
 'теперь',
 'indeed',
 'where',
 'ten',
 'co',
 'about',
 'become',
 'among',
 'de',
 'его',
 'less',
 'whether',
 'else',
 'перед',
 'впрочем',
 'хорошо',
 'below',
 'этой',
 'того',
 'before',
 'under',
 'us',
 'amongst',
 'thick',
 'при',
 'noone',
 'им',
 'до',
 'many',
 'anywhere',
 'этом',
 'is',
 'until',
 'можно',
 'herein',
 'во',
 'others',
 'seemed',
 'somehow',
 '

Проверка на наличие слов `tag` и `link` в списке стоп-слов.

In [13]:
'TAG' in combined_stopwords, 'tag' in combined_stopwords, 'LINK' in combined_stopwords, 'link' in combined_stopwords

(False, False, False, False)

In [14]:
len(combined_stopwords)

469

### Подготовка текстовых и числовых признаков

#### Преобразование текста в числовой формат (TF-IDF)

In [15]:
vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    stop_words=combined_stopwords,
    min_df=2,
    sublinear_tf=True
)

In [16]:
text_features_train = vectorizer.fit_transform(X_train_text)
text_features_test = vectorizer.transform(X_test_text)

#### Масштабирование числовых признаков

In [17]:
numerical_features_train = pd.DataFrame(X_train_num, columns=['emojis', 'newlines', 'whitespaces', 'links', 'tags'])
numerical_features_test = pd.DataFrame(X_test_num, columns=['emojis', 'newlines', 'whitespaces', 'links', 'tags'])

scaler = StandardScaler()
numerical_features_train_scaled = scaler.fit_transform(numerical_features_train)
numerical_features_test_scaled = scaler.transform(numerical_features_test)

#### Объединение текстовых и числовых признаков

In [18]:
X_train = hstack([text_features_train, numerical_features_train_scaled])
X_test = hstack([text_features_test, numerical_features_test_scaled])

#### Дополнительная синтетическая генерация

In [19]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

## Обучение моделей

### Logistic Regression

- Стоп слова NLTK английские + русские  
- Выборка без SMOTE 

In [20]:
model_lr = LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)
model_lr.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,42
,solver,'lbfgs'
,max_iter,500
,multi_class,'deprecated'


### Logistic Regression + SMOTE

- Стоп слова NLTK английские + русские  
- Выборка с SMOTE

In [21]:
model_lr_sm = LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)
model_lr_sm.fit(X_train_resampled, y_train_resampled)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,42
,solver,'lbfgs'
,max_iter,500
,multi_class,'deprecated'


### Random Forest

- Стоп слова NLTK английские + русские  
- Выборка без SMOTE

In [22]:
model_rf = RandomForestClassifier(class_weight='balanced', random_state=42, n_estimators=100)
model_rf.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


### Random Forest + SMOTE

- Стоп слова NLTK английские + русские  
- Выборка с SMOTE

In [23]:
model_rf_sm = RandomForestClassifier(class_weight='balanced', random_state=42, n_estimators=100)
model_rf_sm.fit(X_train_resampled, y_train_resampled)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


### Naive Bayes

- Стоп слова NLTK английские + русские  
- Выборка без SMOTE

Объединение текстов и числовых признаков без масштабирования

In [24]:
X_train_nc = hstack([text_features_train, X_train_num])
X_test_nc = hstack([text_features_test, X_test_num])

Обучение Naive Bayes

In [25]:
model_nb = MultinomialNB()
model_nb.fit(X_train_nc, y_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


### Naive Bayes + SMOTE

- Стоп слова NLTK английские + русские  
- Выборка c SMOTE

Объединение текстов и числовых признаков без масштабирования

In [26]:
X_train_nc = hstack([text_features_train, X_train_num])
X_test_nc = hstack([text_features_test, X_test_num])

In [27]:
smote = SMOTE(random_state=42)
X_train_resampled_nc, y_train_resampled_nc = smote.fit_resample(X_train_nc, y_train)

In [28]:
model_nb_sm = MultinomialNB()
model_nb_sm.fit(X_train_resampled_nc, y_train_resampled_nc)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


## Оценка качества моделей

Примеры тестовых сообщений

In [29]:
test_messages = [
    "Это честное сообщение от пользователя.",
    "🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎",
    "Зарабатывай миллионы **онлайн** прямо сейчас!",
    "Работа на дому, легкий доход. Пиши в личку!",
    "Привет! Как дела? У меня всё отлично.",
    "Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM\n\nhttps://t.me/ancleroyofficial",
    "Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or click the link on my bio",
    "steam gift 50$ - steamcommunity.com/gift-card/pay/50\n@everyone",
    "Давайте **вместе** будем писать про казино в чатах!!! Присоединяйтесь!",
    "Как же надоели эти сообщения про казино",
    "Добрый день. Для подачи документов необходимо пройти регистрацию здесь: stankin.ru",
    "Добрый день. Для подачи документов необходимо пройти регистрацию здесь: https://stankin.ru",
    "Поступление – это почти что казино! Лотерея!",
    "3-4 часа и 8 тысяч твои!  Пиши  https://t.me/rasmuswork1",
    "Выиграл 345к дало x3450\n\nИграл тут: @jet_casino_ibot"
]

Функция для предсказания на новых данных

> Эта же функция находится в `../utils/preprocessing.py`.

In [30]:
def predict_message(text: str, vectorizer, scaler, model) -> tuple[int, list[float]]:
    """
    Предсказывает метку и вероятности для текста с использованием модели машинного обучения.

    Параметры:
        text: (str) - входной текст для анализа
        vectorizer: - векторизатор для преобразования текста
        scaler: - масштабатор числовых признаков
        model: - модель машинного обучения

    Возвращает:
        tuple[int, list[float]] - метка предсказания и вероятности
    """
    # Предварительная обработка текста
    text_preprocessed = preprocess_text(text)

    # Векторизация текста
    text_vector = vectorizer.transform([text_preprocessed])

    # Создание числовых признаков
    numerical_features = pd.DataFrame(
        [[count_emojis(text), count_newlines(text), count_whitespaces(text), count_links(text), count_tags(text)]],
        columns=['emojis', 'newlines', 'whitespaces', 'links', 'tags']
    )
    # Масштабирование числовых признаков
    numerical_features_scaled = scaler.transform(numerical_features)

    # Объединение текстовых и числовых признаков
    features = hstack([text_vector, numerical_features_scaled])

    # Предсказание
    prediction = model.predict(features)
    probabilities = model.predict_proba(features)
    return int(prediction[0]), *probabilities.tolist()

### Logistic Regression

In [31]:
y_pred = model_lr.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.99      0.96      7661
           1       0.98      0.93      0.95      7046

    accuracy                           0.96     14707
   macro avg       0.96      0.96      0.96     14707
weighted avg       0.96      0.96      0.96     14707

Confusion Matrix:
 [[7556  105]
 [ 511 6535]]


In [32]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_lr)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

2025-09-03 02:50:34,519 - INFO - Начало предобработки текста: Это честное сообщение от пользователя.
2025-09-03 02:50:34,521 - INFO - Предобработка завершена. Итоговый текст: это честное сообщение от пользователя
2025-09-03 02:50:34,526 - INFO - Начало предобработки текста: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
2025-09-03 02:50:34,528 - INFO - Предобработка завершена. Итоговый текст: казино онлайн зарабатывай миллионы прямо сейчас
2025-09-03 02:50:34,532 - INFO - Начало предобработки текста: Зарабатывай миллионы **онлайн** прямо сейчас!
2025-09-03 02:50:34,535 - INFO - Предобработка завершена. Итоговый текст: зарабатывай миллионы онлайн прямо сейчас
2025-09-03 02:50:34,544 - INFO - Начало предобработки текста: Работа на дому, легкий доход. Пиши в личку!
2025-09-03 02:50:34,545 - INFO - Предобработка завершена. Итоговый текст: работа на дому легкий доход пиши в личку
2025-09-03 02:50:34,549 - INFO - Начало предобработки текста: Привет! Как дела? У меня всё отлично.
2025

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.9361215, 0.0638785]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 1
Вероятности: [0.0145762, 0.9854238]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 1
Вероятности: [0.4119424, 0.5880576]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 1
Вероятности: [0.0176773, 0.9823227]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.8841986, 0.1158014]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.4

### Logistic Regression + SMOTE

In [33]:
y_pred = model_lr_sm.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.99      0.96      7661
           1       0.98      0.93      0.96      7046

    accuracy                           0.96     14707
   macro avg       0.96      0.96      0.96     14707
weighted avg       0.96      0.96      0.96     14707

Confusion Matrix:
 [[7555  106]
 [ 508 6538]]


In [34]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_lr_sm)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

2025-09-03 02:50:34,633 - INFO - Начало предобработки текста: Это честное сообщение от пользователя.
2025-09-03 02:50:34,634 - INFO - Предобработка завершена. Итоговый текст: это честное сообщение от пользователя
2025-09-03 02:50:34,639 - INFO - Начало предобработки текста: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
2025-09-03 02:50:34,640 - INFO - Предобработка завершена. Итоговый текст: казино онлайн зарабатывай миллионы прямо сейчас
2025-09-03 02:50:34,644 - INFO - Начало предобработки текста: Зарабатывай миллионы **онлайн** прямо сейчас!
2025-09-03 02:50:34,646 - INFO - Предобработка завершена. Итоговый текст: зарабатывай миллионы онлайн прямо сейчас
2025-09-03 02:50:34,649 - INFO - Начало предобработки текста: Работа на дому, легкий доход. Пиши в личку!
2025-09-03 02:50:34,650 - INFO - Предобработка завершена. Итоговый текст: работа на дому легкий доход пиши в личку
2025-09-03 02:50:34,654 - INFO - Начало предобработки текста: Привет! Как дела? У меня всё отлично.
2025

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.9351577, 0.0648423]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 1
Вероятности: [0.0164345, 0.9835655]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 1
Вероятности: [0.4420665, 0.5579335]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 1
Вероятности: [0.0156511, 0.9843489]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.8975201, 0.1024799]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.4

2025-09-03 02:50:34,690 - INFO - Начало предобработки текста: Поступление – это почти что казино! Лотерея!
2025-09-03 02:50:34,691 - INFO - Предобработка завершена. Итоговый текст: поступление это почти что казино лотерея
2025-09-03 02:50:34,695 - INFO - Начало предобработки текста: 3-4 часа и 8 тысяч твои!  Пиши  https://t.me/rasmuswork1
2025-09-03 02:50:34,696 - INFO - Предобработка завершена. Итоговый текст: 34 часа и 8 тысяч твои пиши [LINK]
2025-09-03 02:50:34,699 - INFO - Начало предобработки текста: Выиграл 345к дало x3450

Играл тут: @jet_casino_ibot
2025-09-03 02:50:34,699 - INFO - Предобработка завершена. Итоговый текст: выиграл 345к дало x3450 играл тут [TAG]


Сообщение: Поступление – это почти что казино! Лотерея!
Класс: 0
Вероятности: [0.6786932, 0.3213068]

Сообщение: 3-4 часа и 8 тысяч твои!  Пиши  https://t.me/rasmuswork1
Класс: 1
Вероятности: [0.0578039, 0.9421961]

Сообщение: Выиграл 345к дало x3450

Играл тут: @jet_casino_ibot
Класс: 1
Вероятности: [0.0263081, 0.9736919]



### Random Forest

In [35]:
y_pred = model_rf.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.98      0.96      7661
           1       0.98      0.94      0.96      7046

    accuracy                           0.96     14707
   macro avg       0.96      0.96      0.96     14707
weighted avg       0.96      0.96      0.96     14707

Confusion Matrix:
 [[7512  149]
 [ 424 6622]]


In [36]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_rf)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

2025-09-03 02:50:36,185 - INFO - Начало предобработки текста: Это честное сообщение от пользователя.
2025-09-03 02:50:36,187 - INFO - Предобработка завершена. Итоговый текст: это честное сообщение от пользователя
2025-09-03 02:50:36,207 - INFO - Начало предобработки текста: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
2025-09-03 02:50:36,208 - INFO - Предобработка завершена. Итоговый текст: казино онлайн зарабатывай миллионы прямо сейчас
2025-09-03 02:50:36,223 - INFO - Начало предобработки текста: Зарабатывай миллионы **онлайн** прямо сейчас!
2025-09-03 02:50:36,224 - INFO - Предобработка завершена. Итоговый текст: зарабатывай миллионы онлайн прямо сейчас
2025-09-03 02:50:36,238 - INFO - Начало предобработки текста: Работа на дому, легкий доход. Пиши в личку!
2025-09-03 02:50:36,239 - INFO - Предобработка завершена. Итоговый текст: работа на дому легкий доход пиши в личку
2025-09-03 02:50:36,253 - INFO - Начало предобработки текста: Привет! Как дела? У меня всё отлично.
2025

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.97, 0.03]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 1
Вероятности: [0.0835361, 0.9164639]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.8316867, 0.1683133]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 1
Вероятности: [0.1, 0.9]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.9283343, 0.0716657]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.3656516, 0.6343484]

Со

2025-09-03 02:50:36,376 - INFO - Начало предобработки текста: Поступление – это почти что казино! Лотерея!
2025-09-03 02:50:36,376 - INFO - Предобработка завершена. Итоговый текст: поступление это почти что казино лотерея
2025-09-03 02:50:36,391 - INFO - Начало предобработки текста: 3-4 часа и 8 тысяч твои!  Пиши  https://t.me/rasmuswork1
2025-09-03 02:50:36,392 - INFO - Предобработка завершена. Итоговый текст: 34 часа и 8 тысяч твои пиши [LINK]


Сообщение: Добрый день. Для подачи документов необходимо пройти регистрацию здесь: https://stankin.ru
Класс: 0
Вероятности: [0.8, 0.2]

Сообщение: Поступление – это почти что казино! Лотерея!
Класс: 0
Вероятности: [0.54, 0.46]



2025-09-03 02:50:36,407 - INFO - Начало предобработки текста: Выиграл 345к дало x3450

Играл тут: @jet_casino_ibot
2025-09-03 02:50:36,408 - INFO - Предобработка завершена. Итоговый текст: выиграл 345к дало x3450 играл тут [TAG]


Сообщение: 3-4 часа и 8 тысяч твои!  Пиши  https://t.me/rasmuswork1
Класс: 1
Вероятности: [0.0883474, 0.9116526]

Сообщение: Выиграл 345к дало x3450

Играл тут: @jet_casino_ibot
Класс: 1
Вероятности: [0.0, 1.0]



### Random Forest + SMOTE

In [37]:
y_pred = model_rf_sm.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print('\n\n')

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.98      0.96      7661
           1       0.98      0.94      0.96      7046

    accuracy                           0.96     14707
   macro avg       0.96      0.96      0.96     14707
weighted avg       0.96      0.96      0.96     14707

Confusion Matrix:
 [[7515  146]
 [ 424 6622]]





In [38]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_rf_sm)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

2025-09-03 02:50:37,927 - INFO - Начало предобработки текста: Это честное сообщение от пользователя.
2025-09-03 02:50:37,929 - INFO - Предобработка завершена. Итоговый текст: это честное сообщение от пользователя
2025-09-03 02:50:37,948 - INFO - Начало предобработки текста: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
2025-09-03 02:50:37,949 - INFO - Предобработка завершена. Итоговый текст: казино онлайн зарабатывай миллионы прямо сейчас
2025-09-03 02:50:37,963 - INFO - Начало предобработки текста: Зарабатывай миллионы **онлайн** прямо сейчас!
2025-09-03 02:50:37,964 - INFO - Предобработка завершена. Итоговый текст: зарабатывай миллионы онлайн прямо сейчас
2025-09-03 02:50:37,979 - INFO - Начало предобработки текста: Работа на дому, легкий доход. Пиши в личку!
2025-09-03 02:50:37,980 - INFO - Предобработка завершена. Итоговый текст: работа на дому легкий доход пиши в личку
2025-09-03 02:50:37,993 - INFO - Начало предобработки текста: Привет! Как дела? У меня всё отлично.
2025

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.98, 0.02]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 1
Вероятности: [0.0790328, 0.9209672]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.8487192, 0.1512808]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 1
Вероятности: [0.07, 0.93]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.9151165, 0.0848835]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.3633378, 0.6366622]



2025-09-03 02:50:38,100 - INFO - Начало предобработки текста: Добрый день. Для подачи документов необходимо пройти регистрацию здесь: https://stankin.ru
2025-09-03 02:50:38,101 - INFO - Предобработка завершена. Итоговый текст: добрый день для подачи документов необходимо пройти регистрацию здесь [LINK]
2025-09-03 02:50:38,116 - INFO - Начало предобработки текста: Поступление – это почти что казино! Лотерея!
2025-09-03 02:50:38,117 - INFO - Предобработка завершена. Итоговый текст: поступление это почти что казино лотерея
2025-09-03 02:50:38,131 - INFO - Начало предобработки текста: 3-4 часа и 8 тысяч твои!  Пиши  https://t.me/rasmuswork1


Сообщение: Добрый день. Для подачи документов необходимо пройти регистрацию здесь: https://stankin.ru
Класс: 0
Вероятности: [0.81, 0.19]

Сообщение: Поступление – это почти что казино! Лотерея!
Класс: 1
Вероятности: [0.47, 0.53]



2025-09-03 02:50:38,133 - INFO - Предобработка завершена. Итоговый текст: 34 часа и 8 тысяч твои пиши [LINK]
2025-09-03 02:50:38,147 - INFO - Начало предобработки текста: Выиграл 345к дало x3450

Играл тут: @jet_casino_ibot
2025-09-03 02:50:38,148 - INFO - Предобработка завершена. Итоговый текст: выиграл 345к дало x3450 играл тут [TAG]


Сообщение: 3-4 часа и 8 тысяч твои!  Пиши  https://t.me/rasmuswork1
Класс: 1
Вероятности: [0.0534848, 0.9465152]

Сообщение: Выиграл 345к дало x3450

Играл тут: @jet_casino_ibot
Класс: 1
Вероятности: [0.0, 1.0]



### Naive Bayes

In [39]:
y_pred = model_nb.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print('\n\n')

Classification Report:
               precision    recall  f1-score   support

           0       0.65      1.00      0.79      7661
           1       1.00      0.42      0.59      7046

    accuracy                           0.72     14707
   macro avg       0.82      0.71      0.69     14707
weighted avg       0.82      0.72      0.69     14707

Confusion Matrix:
 [[7652    9]
 [4102 2944]]





In [40]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_nb)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

2025-09-03 02:50:38,196 - INFO - Начало предобработки текста: Это честное сообщение от пользователя.
2025-09-03 02:50:38,197 - INFO - Предобработка завершена. Итоговый текст: это честное сообщение от пользователя
2025-09-03 02:50:38,203 - INFO - Начало предобработки текста: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
2025-09-03 02:50:38,204 - INFO - Предобработка завершена. Итоговый текст: казино онлайн зарабатывай миллионы прямо сейчас
2025-09-03 02:50:38,209 - INFO - Начало предобработки текста: Зарабатывай миллионы **онлайн** прямо сейчас!
2025-09-03 02:50:38,210 - INFO - Предобработка завершена. Итоговый текст: зарабатывай миллионы онлайн прямо сейчас
2025-09-03 02:50:38,215 - INFO - Начало предобработки текста: Работа на дому, легкий доход. Пиши в личку!
2025-09-03 02:50:38,216 - INFO - Предобработка завершена. Итоговый текст: работа на дому легкий доход пиши в личку
2025-09-03 02:50:38,220 - INFO - Начало предобработки текста: Привет! Как дела? У меня всё отлично.
2025

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.9977781, 0.0022219]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 0
Вероятности: [0.9650027, 0.0349973]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.9919941, 0.0080059]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 0
Вероятности: [0.7502888, 0.2497112]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.999412, 0.000588]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 0
Вероятности: [0.992

### Naive Bayes + SMOTE

In [41]:
y_pred = model_nb_sm.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print('\n\n')

Classification Report:
               precision    recall  f1-score   support

           0       0.65      1.00      0.79      7661
           1       1.00      0.43      0.60      7046

    accuracy                           0.72     14707
   macro avg       0.83      0.71      0.69     14707
weighted avg       0.82      0.72      0.70     14707

Confusion Matrix:
 [[7652    9]
 [4050 2996]]





In [42]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_nb_sm)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

2025-09-03 02:50:38,306 - INFO - Начало предобработки текста: Это честное сообщение от пользователя.
2025-09-03 02:50:38,307 - INFO - Предобработка завершена. Итоговый текст: это честное сообщение от пользователя
2025-09-03 02:50:38,312 - INFO - Начало предобработки текста: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
2025-09-03 02:50:38,313 - INFO - Предобработка завершена. Итоговый текст: казино онлайн зарабатывай миллионы прямо сейчас
2025-09-03 02:50:38,318 - INFO - Начало предобработки текста: Зарабатывай миллионы **онлайн** прямо сейчас!
2025-09-03 02:50:38,319 - INFO - Предобработка завершена. Итоговый текст: зарабатывай миллионы онлайн прямо сейчас
2025-09-03 02:50:38,324 - INFO - Начало предобработки текста: Работа на дому, легкий доход. Пиши в личку!
2025-09-03 02:50:38,324 - INFO - Предобработка завершена. Итоговый текст: работа на дому легкий доход пиши в личку
2025-09-03 02:50:38,329 - INFO - Начало предобработки текста: Привет! Как дела? У меня всё отлично.
2025

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.9975711, 0.0024289]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 0
Вероятности: [0.9625256, 0.0374744]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.9913301, 0.0086699]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 0
Вероятности: [0.7338732, 0.2661268]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.9993999, 0.0006001]



2025-09-03 02:50:38,335 - INFO - Предобработка завершена. Итоговый текст: discover the hidden secrets of the digital market that top traders dont want you to know im seeking five motivated individuals who are committed to earning over 100k weekly in the digital market once you start seeing profits ill require just 15 of your earnings as my fee please note im only interested in working with five serious and dedicated people should send me a direct message or ask me how via telegram [LINK]
2025-09-03 02:50:38,339 - INFO - Начало предобработки текста: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or click the link on my bio
2025-09-03 02:50:38,3

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 0
Вероятности: [0.9923029, 0.0076971]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or click the link on my bio
Класс: 0
Вероятности: [0.9951334, 0.0048666]

Сообще

2025-09-03 02:50:38,363 - INFO - Начало предобработки текста: Добрый день. Для подачи документов необходимо пройти регистрацию здесь: https://stankin.ru
2025-09-03 02:50:38,364 - INFO - Предобработка завершена. Итоговый текст: добрый день для подачи документов необходимо пройти регистрацию здесь [LINK]
2025-09-03 02:50:38,368 - INFO - Начало предобработки текста: Поступление – это почти что казино! Лотерея!
2025-09-03 02:50:38,369 - INFO - Предобработка завершена. Итоговый текст: поступление это почти что казино лотерея
2025-09-03 02:50:38,373 - INFO - Начало предобработки текста: 3-4 часа и 8 тысяч твои!  Пиши  https://t.me/rasmuswork1
2025-09-03 02:50:38,373 - INFO - Предобработка завершена. Итоговый текст: 34 часа и 8 тысяч твои пиши [LINK]
2025-09-03 02:50:38,377 - INFO - Начало предобработки текста: Выиграл 345к дало x3450

Играл тут: @jet_casino_ibot


Сообщение: Добрый день. Для подачи документов необходимо пройти регистрацию здесь: https://stankin.ru
Класс: 0
Вероятности: [0.9999984, 1.6e-06]

Сообщение: Поступление – это почти что казино! Лотерея!
Класс: 0
Вероятности: [0.9996401, 0.0003599]

Сообщение: 3-4 часа и 8 тысяч твои!  Пиши  https://t.me/rasmuswork1
Класс: 0
Вероятности: [0.9859109, 0.0140891]



2025-09-03 02:50:38,378 - INFO - Предобработка завершена. Итоговый текст: выиграл 345к дало x3450 играл тут [TAG]


Сообщение: Выиграл 345к дало x3450

Играл тут: @jet_casino_ibot
Класс: 0
Вероятности: [0.9095946, 0.0904054]



## Сохранение результатов

In [43]:
with open('../models/vectorizer.pkl', 'wb') as vec_file:
    pickle.dump(vectorizer, vec_file)

with open('../models/scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

print("Векторизатор и масштабатор сохранены!")

Векторизатор и масштабатор сохранены!


In [44]:
with open('../models/lr_model.pkl', 'wb') as lr_model_file:
    pickle.dump(model_lr, lr_model_file)

print("Модель логической регрессии без семплирования сохранена!")

Модель логической регрессии без семплирования сохранена!


In [45]:
with open('../models/lr_sm_model.pkl', 'wb') as lr_sm_model_file:
    pickle.dump(model_lr_sm, lr_sm_model_file)

print("Модель логической регрессии со семплированием сохранена!")

Модель логической регрессии со семплированием сохранена!


In [46]:
with open('../models/rf_model.pkl', 'wb') as rf_model_file:
    pickle.dump(model_rf, rf_model_file)

print("Модель случайного леса без семплирования сохранена!")

Модель случайного леса без семплирования сохранена!


In [47]:
with open('../models/rf_sm_model.pkl', 'wb') as rf_sm_model_file:
    pickle.dump(model_rf_sm, rf_sm_model_file)

print("Модель случайного леса со семплированием сохранена!")

Модель случайного леса со семплированием сохранена!


In [48]:
with open('../models/nb_model.pkl', 'wb') as nb_model_file:
    pickle.dump(model_nb, nb_model_file)

print("Модель Naive Bayes без семплирования сохранена!")

Модель Naive Bayes без семплирования сохранена!


In [49]:
with open('../models/nb_sm_model.pkl', 'wb') as nb_sm_model_file:
    pickle.dump(model_nb_sm, nb_sm_model_file)

print("Модель Naive Bayes со семплированием сохранена!")

Модель Naive Bayes со семплированием сохранена!
